In [ ]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import os
import shutil

In [ ]:
import pandas as pd
gse = pd.read_csv(R"D:\Project\gutDBase\SRR_Acc_List\human\GSE_PRJ_list.txt", sep='\t', index_col=None, header=None)

In [ ]:
gse

In [ ]:
gse[gse[gse.columns[2]] == 'Yes']

In [ ]:
GSE_list = list(gse[gse[gse.columns[2]] == 'Yes'][gse.columns[0]][8:])

In [ ]:
len(GSE_list)

In [ ]:
chromedriver_path = R"D:\Tool\chromedriver.exe"
download_dir = R"D:\Project\gutDBase\metadata\human"
os.makedirs(download_dir, exist_ok=True)

service = Service(chromedriver_path)

chrome_options = Options()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
}
chrome_options.add_experimental_option("prefs", prefs)

In [ ]:
ChromeBrowser = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
ChromeBrowser.implicitly_wait(20)

In [ ]:
GSE_PRJ_list = []

In [ ]:
for GSE in GSE_list:
    ChromeBrowser.get(
        fr'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={GSE}')
    sleep(10)
    links = ChromeBrowser.find_elements(By.TAG_NAME, "a")
    bioproject_links = [link for link in links if "PRJ" in link.text]
    BioProject = bioproject_links[0].text
    print(f'{GSE}\t{BioProject}')
    SRA_Run_Selector_link = [
        link for link in links if "SRA Run Selector" in link.text]
    if len(SRA_Run_Selector_link) == 0:
        GSE_PRJ_list.append((GSE, BioProject, 'No'))
        print("No SRA Run Selector link found.")
    else:
        SRA_Run_Selector_link = SRA_Run_Selector_link[0]
        GSE_PRJ_list.append((GSE, BioProject, 'Yes'))
        print("SRA Run Selector link found.")
        SRA_Run_Selector_link.click()
        download_button = ChromeBrowser.find_element(
            By.CSS_SELECTOR, '#t-rit-all')
        download_button.click()
        sleep(5)
        shutil.move(os.path.join(download_dir, r'SraRunTable.csv'),
                    os.path.join(download_dir, fr'{BioProject}.csv'))

In [ ]:
GSE_PRJ_list